# Comparing Solar Generation to Usage Data

In order to calculate the payback period of our solar installation, we need to first determine what it will generate.  We have solar radiation data from our weather station in watts per square meter with 5-minute resolution.  We have usage data from the utility in kilowatt-hours with 15-minute resolution.  We can use these to calculate how much solar we would have generated for a given time period in order to calculate the amount of power usage that would be offset by solar.

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import pytz

## Getting our data

We can get our solar data from our weather station.  AmbientWeather provides a csv export feature which is documented [here](https://ambientweather.com/faqs/question/view/id/1807/).  We selected a year of data for the best representation.

We can get our usage data from the utility.  The process for getting a csv of usage data is outlined in the [Green Button Download My Data](https://www.pge.com/en/save-energy-and-money/energy-usage-and-tips/understand-my-usage/energy-data-hub.html) section of [this document](https://www.pge.com/en/save-energy-and-money/energy-usage-and-tips/understand-my-usage/energy-data-hub.html).

In [ ]:
solar_data = pd.read_csv('../data/raw/Solar ROI - ambient-weather-20230203-20240130.csv')
usage_data = pd.read_csv('../data/raw/Solar ROI - pge_electric_interval_data_0627600496_2023-01-30_to_2024-01-30.csv')

We need to normalize the times and dates in our datasets.  Our solar data is in UTC and the `Date` column is a complete timestamp.  Each record in solar data is a measurement at that point in time.

On the other hand, the usage data is already in local time, but each record represents a time window.  The column `DATE` represents the year-month-day and `START TIME` and `END TIME` represent the start and end of the time window, expressed in 24-hour clock without leading zeroes.

In [ ]:
# Convert the 'date' column in both datasets to datetime objects and localize to America/Los_Angeles timezone
solar_data['Date'] = pd.to_datetime(solar_data['Date'], utc=True, errors='coerce').dt.tz_convert('America/Los_Angeles')
usage_data['DATE'] = pd.to_datetime(usage_data['DATE'], errors='coerce').dt.tz_localize('America/Los_Angeles', ambiguous='NaT')

# Convert the 'START TIME' and 'END TIME' columns to datetime objects
usage_data['START TIME'] = pd.to_datetime(usage_data['START TIME'], format='%H:%M').dt.time
usage_data['END TIME'] = pd.to_datetime(usage_data['END TIME'], format='%H:%M').dt.time

## Calculating Solar Generation

Now we have solar generation for points in time expressed in watts per square meter and we have usage in 15 minute time windows expressed in kilowatt hours.  We need to calculate how much solar energy we would generate in each 15 minute window.  We'll start by averaging the generation using all solar datapoints within the usage time window.  Of course, we don't get to use all of that radiation, so we'll apply some coefficients.  The solar panels we're considering are 20.3% efficient.  The rest of the system (inverter, optimizers, etc) are about 95% efficent.  We'll need to adjust for that.  We then need the total area of the photovoltaic cells.  Each panel is about 1.97 square meters large.  We'll have 16 panels, so we use 31.52 square meters as our area.

Using these coefficients, we can calculate the amount of energy generated in each time window and add that to our usage data.  Once we have that, we can calculate our offsets.

We calculate the adjusted usage by subtracting the solar generation from the original usage. If the result is negative (i.e., the solar generation is greater than the usage), we set it to zero. This ensures that we don't have negative adjusted usage.

We calculate the remaining solar generation that can be sold to the grid by subtracting the adjusted usage from the solar generation. If the result is negative (i.e., the adjusted usage is greater than the solar generation), we set it to zero. This ensures that we don't have negative grid sell values.

In [ ]:
solar_panel_efficiency = 0.203
system_efficiency = 0.95
solar_panel_surface_area = 31.52  # in square meters

def calculate_solar_generated(row):
    relevant_solar_data = solar_data[
        (solar_data['Date'].dt.date == row['DATE'].date()) &
        (solar_data['Date'].dt.time >= row['START TIME']) &
        (solar_data['Date'].dt.time <= row['END TIME'])
    ]
    
    average_solar_radiation = relevant_solar_data['Solar Radiation (W/m^2)'].mean()
    usage_duration_hours = (pd.to_datetime(row['END TIME'].strftime('%H:%M:%S')) - pd.to_datetime(row['START TIME'].strftime('%H:%M:%S'))).seconds / 3600
    
    # Adjusted solar_generated with efficiency considerations and surface area
    solar_generated = average_solar_radiation * solar_panel_efficiency * system_efficiency * solar_panel_surface_area * usage_duration_hours
    
    return solar_generated / 1000  # Convert from Wh to kWh

# Apply the function to create the 'solar_generated' column with tqdm_notebook for progress bar
tqdm.pandas()
usage_data['solar_generated'] = usage_data.progress_apply(calculate_solar_generated, axis=1)

# Calculate usage_adjusted and grid_sell columns
usage_data['usage_adjusted'] = usage_data.apply(lambda row: max(row['USAGE'] - row['solar_generated'], 0), axis=1)
usage_data['grid_sell'] = usage_data.apply(lambda row: max(row['solar_generated'] - row['usage_adjusted'], 0), axis=1)

# Save our progress
usage_data.to_csv('../data/processed/Solar ROI - usage_data_with_generation.csv')